In [1]:
import os

In [2]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one\\Experimentation'

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one'

In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [25]:
# ENTITY

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzipped_data_dir: Path
    all_schema: dict

In [11]:
#  Configuration Manager imports

from portfolio_project_one.constants import *
from portfolio_project_one.utils.common import read_yaml, create_directories


In [12]:
#  Configuration Manager class

class ConfigurationManager:

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifact_root])


    def get_data_validated_config(self) -> DataValidationConfig:                    # only copy this function to Configuration Manager
                                                                                                # kyuki upar ka function toh same hi hai
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzipped_data_dir = config.unzipped_data_dir,
            all_schema = schema
        )

        return data_validation_config
        

In [13]:
#  Components imports

import os
from portfolio_project_one import logger

In [37]:
#  Components

class DataValidation:

    def __init__(
        self,
        config: DataValidationConfig
    ):
        self.config = config

    def validate_all_columns(self) -> bool:   # this only checks if the column name exits, we can also add validaate datatype
        try:
            validation_status = None

            df = pd.read_csv(self.config.unzipped_data_dir)
            all_columns = list(df.columns)

            all_schema = self.config.all_schema

            for column in all_columns:
                if column not in all_schema.keys():   
                    validation_status = False
                    #  this means we should not go ahead
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'''******{column}******* is an anmoly, its not in schema.yaml, deal with this first then run pipeline again
                                    VALIDATION STATUS:  {validation_status}.''')

                        logger.info(f'Validation has failed for column : {column}, Check svhema.yaml, Status File and DataFrame.')
                    break   # break the loop so that we dont go ahead and end up getting a pass status at the end
                    
                elif str(df[column].dtype) == all_schema[column]:       # after or we check for datatype(personal code)
                    validation_status = True                           # column exists and datatype is satisfied
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'''VALIDATION STATUS:  {validation_status}.''')
                    logger.info(f'Validation is Succesfull')

                else:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'''******{column}******* is an anmoly, its datatype is a mismatch, deal with this first then run pipeline again
                                    VALIDATION STATUS:  {validation_status}.''')

                        logger.info(f'Validation has failed for column : {column}, Check schema.yaml, Status File and DataFrame.')
                    break

            return validation_status

        except Exception as e:
            logger.info(f'Error Occured while Data_Validation')
            raise e
            

In [44]:
#  Try rough pipeline

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validated_config()
    data_validation = DataValidation(config= data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-10-07 15:57:10,823: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-07 15:57:10,826: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-07 15:57:10,832: INFO: common: yaml file: schema.yaml loaded succesfully]
[2023-10-07 15:57:10,834: INFO: common: Created directory at : artifacts]
[2023-10-07 15:57:10,836: INFO: common: Created directory at : artifacts/data_validation]
[2023-10-07 15:57:10,841: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,841: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,853: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,856: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,859: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,861: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,863: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,866: INFO: 898625526: Validation is Succesfull]
[2023-10-07 15:57:10,868: IN

In [41]:
#  Now all we need to do is to convert this pipeline to modular code